## <center> **Livrable n°1 : Classification binaire** </center>

‎ 

Réalisé par le **groupe n°2** :
- BERTHO Lucien
- BOSACKI Paul
- GAURE Warren
- GRENOUILLET Théo
- VALLEMONT Hugo


‎

---


### **Sommaire**

1. [Mise en contexte](#contexte)
2. [Objectif du livrable](#objectif)
3. [Importation des bibliothèques](#import)
4. [Préparation et chargement des données](#load)
5. [Exploration des données](#exploration)
6. [Configuration de l'environnement](#configuration)
7. [Choix de l'architecture](#architecture)
8. [Réalisation du modèle](#modele)
9.  [Entraînement et évaluation du modèle](#train)
10. [Amélioration du modèle](#amelioration)
11. [Modèle final](#final)
12. [Conclusion](#conclusion)

‎ 

---

### 1. <a id='contexte'>Mise en contexte</a>

L’entreprise TouNum est spécialisée dans la numérisation de documents, qu’il s’agisse de textes ou d’images. Ses services sont particulièrement sollicités par des entreprises cherchant à transformer leur base documentaire papier en fichiers numériques exploitables. Aujourd’hui, TouNum souhaite aller plus loin en enrichissant son offre avec des outils basés sur le Machine Learning.

En effet, certains clients disposent d’un volume considérable de documents à numériser et expriment un besoin croissant pour des solutions de catégorisation automatique. Une telle innovation leur permettrait d’optimiser le traitement et l’exploitation de leurs données numérisées. Toutefois, TouNum ne dispose pas en interne des compétences nécessaires pour concevoir et mettre en place ces technologies.

C’est dans ce cadre que notre équipe de spécialistes en Data Science du CESI est sollicitée. Notre mission consiste à développer une première solution intégrant du captioning automatique : un système capable d’analyser des photographies et de générer une légende descriptive de manière autonome.

Heureusement, TouNum possède déjà plusieurs milliers d’images annotées, ce qui constituera une ressource précieuse pour entraîner les modèles de Machine Learning à partir d’un apprentissage supervisé.

---

### 2. <a id='objectif'>Objectif du livrable</a>

TouNum souhaite automatiser la sélection des photos destinées à l'annotation. Ce livrable propose une méthode de classification basée sur les réseaux de neurones pour filtrer les images qui ne sont pas des photos. La solution reposera sur l'architecture de réseau retenue en fonction des résultats obtenus.

---

### 3. <a id='import'>Importation des bibliothèques</a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import collections
import os
import datetime
import keras_tuner as kt

from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight

---

### 4. <a id='load'>Préparation et chargement des données</a>

Une fois les bibliothèques importées, nous pouvons commencer à préparer le terrain en amont et charger les données pour qu'elles puissent être utilisées dans notre pipeline.

#### 4.1. <a>Filtrage des données</a>

Dans un premier temps, nous devons veiller à ce que nous aillons bien reçu uniquement des images, c'est-à-dire vérifier qu'il n'y ait pas d'intrus comme des fichiers textes ou autres.

In [ ]:
dataset_directory = "dataset_livrable_1/"

In [ ]:
def is_image(filename):
    try:
        with Image.open(filename) as img:
            img.verify()
        return True
    except Exception:
        return False

def remove_non_images(directory):
    for folder, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(folder, file)

            if not is_image(file_path):
                print(f"Suppression de {file_path}")
                os.remove(file_path)
                
remove_non_images(dataset_directory)

──────────────────────────────────────────────────

#### 4.2. <a>Vérification des images</a>

Ensuite, nous devons nous assurer du bon état des images reçues, cela signifie vérifier si elles n'ont pas été corrompues ou mal formatées.

In [ ]:
def is_valid_image(path):
    try:
        img_raw = tf.io.read_file(path)
        _ = tf.image.decode_image(img_raw, channels=3)
        return (path, True)
    except Exception:
        return (path, False)

def clean_corrupted_images(directory, extensions=("jpg", "jpeg", "png"), max_workers=8):
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extensions):
                image_paths.append(os.path.join(root, file))

    print(f"Scan de {len(image_paths)} images dans {directory}")

    corrupted_count = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(is_valid_image, path) for path in image_paths]
        for future in as_completed(futures):
            path, is_valid = future.result()
            if not is_valid:
                try:
                    os.remove(path)
                    corrupted_count += 1
                except Exception as e:
                    print(f"Erreur de suppression {path} : {e}")

    print(f"Vérification terminée : {corrupted_count} image(s) corrompue(s) supprimée(s).")
    

clean_corrupted_images(dataset_directory)

──────────────────────────────────────────────────

#### 4.3. <a>Gestion des logs</a>

Nous créons ici un dossier qui va nous permettre de stocker les logs qui seront utilisés par [TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard).

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

──────────────────────────────────────────────────

#### 4.4. <a>Chargement des images</a>

Les images doivent être séparées en deux ensembles : un pour l'entraînement du modèle, l'autre pour son évaluation.

In [ ]:
image_h = 128
image_w = 128
batch_s = 16

train_set, test_set = keras.utils.image_dataset_from_directory(
    dataset_directory,
    label_mode = "int",
    batch_size = batch_s,
    image_size = (image_h, image_w),
    seed = 42,
    validation_split = 0.2,
    subset = "both"
)

---

### 5. <a id='exploration'>Exploration des données</a>

Maintenant que les données ont pu être préparées et chargées, nous pouvons nous intéresser de plus près à elles en les explorant.

#### 5.1. <a>Nom des classes</a>

In [ ]:
class_names = train_set.class_names
print(f"Classes détectées : {class_names}")

Nous retrouvons bien les 5 classes attendues.

──────────────────────────────────────────────────

#### 5.2. <a>Répartition des données</a>

Nous regardons maintenant la répartition des données entre les classes.

In [ ]:
def print_class_distribution(dataset, name):
    label_counts = collections.Counter(label.numpy() for _, label in dataset.unbatch())

    classes = {0: "peintures", 1: "photos", 2: "schémas", 3: "croquis", 4: "textes scannés"}

    total = sum(label_counts.values())
  
    labels = []
    counts = []
    percentages = []

    for label_id in sorted(label_counts):
        class_name = classes.get(label_id, f"Classe inconnue ({label_id})")
        count = label_counts[label_id]
        labels.append(class_name)
        counts.append(count)
        percentages.append(count / total * 100)

    plt.figure(figsize=(10, 6))
    bars = plt.bar(labels, counts)

    for bar, pct in zip(bars, percentages):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f"{pct:.1f}%", ha='center', va='bottom')

    plt.title(f"Répartition des classes ({name}_set - {total} images)")
    plt.xlabel("Classe")
    plt.ylabel("Nombre d'images")
    plt.xticks(rotation=15)
    plt.tight_layout()
    plt.show()

In [ ]:
print_class_distribution(train_set, "train")

In [ ]:
print_class_distribution(test_set, "test")

Comme on peut l'observer sur les histogrammes générés, la répartition des données entre les différentes classes est déséquilibrée dans les deux ensembles. Bien que cela ne soit pas gênant dans le set de test car nous souhaitons avoir des conditions proches de la réalité, cela peut poser problème lors de l'entraînement de notre modèle, qui risque d'être biaisé envers les classes majoritaires.

──────────────────────────────────────────────────

#### 5.3. <a>Taille des données</a>

Nous affichons maintenant la taille des données, information pouvant être utile par la suite.

In [ ]:
images, labels = next(iter(train_set.take(1)))
print(f"Tensor des images : {images.shape}")
print(f"Tensor des labels : {labels.shape}")

──────────────────────────────────────────────────

#### 5.4. <a>Affichage des images</a>

Enfin, nous affichons quelques images pour voir plus en détail ce à quoi nous avons affaire.

In [ ]:
plt.figure(figsize = (8, 8))
for images, labels in train_set.take(10):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i].numpy()])
        plt.axis("off")

---

### 6. <a id='configuration'>Configuration de l'environnement</a>

Pour optimiser les performances des calculs, nous allons configurer les données à l’aide de deux fonctions : `Dataset.cache` et `Dataset.prefetch`.  
- [`Dataset.cache`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) stocke les données en mémoire pour éviter les accès répétés au disque.  
- [`Dataset.prefetch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) permet de traiter un élément en arrière-plan pendant l'entraînement ou l'évaluation.  

En combinant ces techniques, nous réduirons significativement le temps de traitement et la charge computationnelle.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size = AUTOTUNE)

---

### 7. <a id='architecture'>Choix de l'architecture</a>

Les **Convolutional Neural Networks (CNN)** sont devenus l’architecture de référence pour les tâches de classification d’images, notamment en classification multi-classes. Leur efficacité repose sur leur capacité à exploiter la structure spatiale locale des images à travers des opérations de convolution, permettant ainsi une extraction hiérarchique des caractéristiques visuelles (bords, formes, textures…).

Historiquement, LeCun et al. (1998) ont démontré la pertinence des CNN dans la reconnaissance de chiffres manuscrits avec LeNet-5. Cette approche a été fortement étendue avec AlexNet (Krizhevsky et al., 2012), qui a surpassé toutes les autres méthodes sur le défi ImageNet, impliquant la classification dans 1000 classes différentes. Depuis, des architectures plus profondes comme VGG, ResNet ou EfficientNet ont confirmé la domination des CNN dans ce domaine (Rawat & Wang, 2017).

De nombreux frameworks modernes (TensorFlow, PyTorch) proposent des implémentations standardisées de CNN pour la classification multi-classes, et les performances obtenues dépassent largement celles des méthodes classiques (SVM, k-NN, etc.) sur des datasets variés.

En résumé, le choix d’un CNN est justifié par :
- Sa capacité à apprendre automatiquement des représentations visuelles pertinentes
- Son efficacité démontrée sur des benchmarks multi-classes (ex : CIFAR-10, ImageNet)
- Sa large adoption dans la recherche et l’industrie pour les tâches de vision par ordinateur

**Sources**
1. Lecun, Yann & Bottou, Leon & Bengio, Y. & Haffner, Patrick. (1998). Gradient-Based Learning Applied to Document Recognition. Proceedings of the IEEE. 86. 2278 - 2324. 10.1109/5.726791.
2. Krizhevsky, Alex & Sutskever, Ilya & Hinton, Geoffrey. (2012). ImageNet Classification with Deep Convolutional Neural Networks. Neural Information Processing Systems. 25. 10.1145/3065386.
3. Rawat, Waseem & Wang, Zenghui. (2017). Deep Convolutional Neural Networks for Image Classification: A Comprehensive Review. Neural Computation. 29. 2352-2449. 10.1162/NECO_a_00990.

---

### 8. <a id='modele'>Réalisation du modèle</a>

Maintenant que le choix de l'architecture est fait, nous pouvons commencer à créer le modèle que nous allons utiliser pour classifier les images envoyées par l'entreprise.

#### 8.1. <a>Création du modèle de base</a>

Notre modèle sera structuré autour des blocs suivants :  
- Une **couche de rescaling** pour normaliser les valeurs des composantes RGB des pixels dans l'intervalle `[0;1]`.  
- Une **première convolution** avec 16 filtres de taille 3x3 (`Conv2D`), suivie d'un **max pooling** pour réduire la dimension spatiale.  
- Une **seconde convolution** utilisant 32 filtres de taille 3x3.  
- Une **troisième convolution** avec 64 filtres de taille 3x3.  
- Une **transformation en vecteur** via une opération d'aplatissement (`Flatten`).  
- Une **couche dense** de 128 unités pour capturer les caractéristiques extraites.  
- Enfin, une **sortie entièrement connectée** avec 5 unités, correspondant aux classes des données.  

In [ ]:
num_classes = len(class_names)

def create_model(use_dropout = False, show_summary = True, lr = 0.001, activation = 'relu'):
    model = Sequential()
    
    model.add(layers.Rescaling(1./255))
    model.add(layers.Conv2D(16, (3, 3), padding = 'same', activation = activation))
    model.add(layers.MaxPooling2D((2, 2)))
    
    if use_dropout:
        model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv2D(32, (3, 3), padding = 'same', activation = activation))
    model.add(layers.Conv2D(64, (3, 3), padding = 'same', activation = activation))
    
    if use_dropout:
        model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation = activation))
    
    if use_dropout:
        model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(num_classes, activation = 'softmax'))
    
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr),
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
        metrics = ['accuracy']
    )
    
    if show_summary:
        model.summary()
    
    return model

L'optimiseur [`Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) est choisi pour sa capacité d'adaptation et sa rapidité de convergence. La fonction de perte [`SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy), quant à elle, est utilisée car jugée plus efficace en mémoire que d'autres fonctions et bien adaptée à la classification multi-classes.

In [ ]:
model = create_model()

──────────────────────────────────────────────────

#### 8.2. <a>Tuning des hyperparamètres</a>

Dans cette section, nous allons effectuer le tuning des hyperparamètres, c’est-à-dire le réglage manuel ou automatique des paramètres qui contrôlent le comportement du modèle, comme le taux d’apprentissage, la taille des couches ou la taille des batchs.
Cela permet d’optimiser les performances du modèle en trouvant la combinaison de paramètres qui offre les meilleurs résultats sur les données de validation.

On cherche à optimiser les parametres suivant : l'utilisation ou non de **dropout**, le **pas d'apprentissage**, quelle **fonction d'activation** donne de meilleur résultat entre relu et tanh puis le **nombre d'epoch**.

On utilise l'algorythme **Hyperband** pour chercher ces hyperparamètres.

In [ ]:
def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = create_model(
        use_dropout=dropout,
        show_summary=False,
        lr=lr,
        activation=activation
    )
    return model

tuner = kt.Hyperband(
    hypermodel = build_model,
    objective = 'val_accuracy',
    max_epochs = 10,
    factor = 3,
    directory = 'hyperband',
    project_name = 'hyperband_test'
)
stop_early = keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5,
    restore_best_weights = True
)

In [ ]:
train_size = int(0.8 * len(train_set))  # 80% for training
val_size = len(train_set) - train_size  # 20% for validation

train_dataset = train_set.take(train_size)
val_dataset = train_set.skip(train_size)

tuner.search(train_dataset, validation_data=val_dataset, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_set, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_set, epochs=best_epoch, validation_split=0.2)

---

### 9. <a id='train'>Entraînement et évaluation du modèle</a>

Avec le modèle créé, nous pouvons désormais procéder à son entraînement et à son évaluation avec les ensembles de données à notre disposition.

#### 9.1. <a>Graphiques</a>

Nous utilisons des graphiques afin de visualiser les courbes d’accuracy pour suivre en temps réel les performances du modèle sur les données d’entraînement et de validation. Cela permet de détecter rapidement les signes de surapprentissage ou de sous-apprentissage.

Avant de générer les graphiques, nous créons les deux callbacks suivants :
- [`TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard?hl=en) : Il permet de visualiser en temps réel l’évolution des métriques et du modèle, facilitant l’analyse et le suivi de l’entraînement.
- [`ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint?hl=en) : Ce callback permet de sauvegarder automatiquement le meilleur modèle au cours de l’entraînement, évitant ainsi de perdre les meilleures performances.

In [ ]:
callbacks = []

In [ ]:
tensorboard_callback = TensorBoard(
    log_dir = log_dir,
    histogram_freq = 1
)

checkpoint_callback = ModelCheckpoint(
    filepath = 'checkpoints/best_model.keras',
    monitor = 'val_accuracy',
    save_best_only = True,
    save_weights_only = False,
    mode = 'max',
    verbose = 1
)

callbacks.append(tensorboard_callback)
callbacks.append(checkpoint_callback)

Comme nous l'avons observé lors de la phase d'exploration des données, il y a un déséquilibre notable dans la répartition des données entre les classes des deux ensembles. Pour palier à ce problème lors de l'entraînement de notre modèle, nous allons ajouter des poids aux classes, qui seront générés grâce à la méthode [`compute_class_weight`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html) de la librairie [scikit-learn](https://scikit-learn.org/stable/index.html). Ceci aura pour effet de renforcer l’importance des classes minoritaires lors de l’apprentissage, afin que le modèle ne privilégie pas uniquement les classes majoritaires.

In [ ]:
y_train = np.array([label.numpy() for _, label in train_set.unbatch()])
class_weights = compute_class_weight(class_weight = "balanced", classes = np.unique(y_train), y = y_train)
weights_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}

In [ ]:
def train_model(model, train_set = train_set, test_set = test_set, epochs = 10):

    history = model.fit(
        train_set,
        validation_data = test_set,
        epochs = epochs,
        callbacks = callbacks,
        class_weight = weights_dict
    )
    
    accuracy = history.history['accuracy']
    validation_accuracy = history.history['val_accuracy']
    
    epochs_range = range(epochs) if epochs == 10 else range(len(accuracy))
    
    loss = history.history['loss']
    validation_loss = history.history['val_loss']
    
    plt.figure(figsize = (16, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
    plt.plot(epochs_range, validation_accuracy, label = 'Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title("Training and Validation Accuracy")
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label = 'Training Loss')
    plt.plot(epochs_range, validation_loss, label = 'Validation Loss')
    plt.legend(loc='upper right')
    plt.title("Training and Validation Loss")
    
    plt.show()

In [ ]:
train_model(model)

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

──────────────────────────────────────────────────

#### 9.2. <a>Matrice de confusion</a>

Nous utilisons une matrice de confusion pour évaluer plus finement les performances d’un modèle en montrant les erreurs de classification pour chaque classe. Elle met en évidence les classes confondues et aide à cibler les axes d’amélioration.

In [ ]:
X_test = []
y_true = []

for images, labels in test_set:
    X_test.append(images)
    y_true.append(labels)

X_test = np.concatenate(X_test)
y_true = np.concatenate(y_true)

def display_matrix(model, X_test = X_test, y_true = y_true, class_names = class_names):
    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis = 1)
    cm = confusion_matrix(y_true, y_pred)
    display = ConfusionMatrixDisplay(cm, display_labels = class_names)
    display.plot(cmap = plt.cm.Blues)
    plt.title("Matrice de confusion")
    plt.xticks(rotation = 45)
    plt.show()

In [ ]:
display_matrix(model)

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

#### 9.3. <a>TensorBoard</a>

Nous utilisons TensorBoard pour visualiser de manière interactive l’entraînement du modèle, en suivant l’évolution des métriques, la structure du réseau et d’autres informations utiles pour le debug et l’optimisation.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

---

### 10. <a id='amelioration'>Amélioration du modèle</a>

Afin de palier au surapprentissage observé et d’améliorer la généralisation du modèle, plusieurs techniques de régularisation ont été retenues :

- [Data Augmentation](#augmentation): Cette technique consister à générer artificiellement de nouvelles images en appliquant des transformations aléatoires aux données existantes. Elle permet d'améliorer la généralisation du modèle en le rendant plus robuste aux variations comme l’orientation, la luminosité ou le zoom.

- [Dropout](#dropout) : Cette méthode consiste à désactiver aléatoirement un certain pourcentage de neurones à chaque itération lors de l'entraînement. Cela empêche le modèle de devenir trop dépendant de certaines connexions et encourage l'apprentissage de représentations plus robustes. Une valeur typique se situe entre 0.2 et 0.5 selon la complexité du réseau.

- [Early-Stopping](#early-stopping) : Cette technique permet d'arrêter automatiquement l'entraînement lorsque la performance sur l’ensemble de validation commence à se dégrader. Elle évite d’entraîner le modèle trop longtemps, ce qui pourrait mener à un surajustement aux données d’entraînement. Un paramètre clé est la `patience`, qui définit le nombre d’époques d'attente avant d'interrompre l'entraînement si aucune amélioration n'est observée.

En testant et, potentiellement, combinant ces différentes approches, nous parviendrons à obtenir un modèle plus stable, robuste, et capable de mieux généraliser sur des données non vues.

──────────────────────────────────────────────────

#### 10.1. <a id='augmentation'>Data Augmentation</a>

Pour cette approche, nous allons appliquer des transformations aux données d'entraînement, comme un retournement aléatoire, une rotation de 10% et d’un zoom vertical de 10%. Les données de test restent inchangées pour permettre au modèle de pouvoir faire des prédictions sur un cas réel.

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip(input_shape = (image_h, image_w, 3), mode = 'horizontal_and_vertical'),
    layers.RandomRotation(factor = 0.1, fill_mode = 'nearest'),
    layers.RandomZoom(height_factor = 0.1, fill_mode = 'nearest'),
])

augmented_train_set = train_set.map(lambda x, y: (data_augmentation(x, training = True), y))

En procédant à l'entraînement de ce modèle comme nous l'avons vu auparavant, nous obtenons les graphiques suivants :

##### <center style="color:yellow;"> **Insérer graphiques** </center>

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests (avec le dataset du WKS2)**
- Méthode efficace pour gérer le surapprentissage
- Les courbes d'accuracy augmentent ensemble de manière cohérente
- Pas de divergence apparente
- Très bonne capacité de généralisation vu que la courbe de Validation Accuracy est souvent au-dessus de celle de la Training Accuracy

##### <center style="color:yellow;"> **Insérer matrice de confusion** </center>

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

──────────────────────────────────────────────────

#### 10.2. <a id='dropout'>Dropout</a>

Pour cette approche, nous ajoutons les couches de Dropout suivantes :
- La première avec un taux de 25% après le MaxPooling
- La seconde de 25% aussi après la troisième couche de convolution
- La dernière avec un taux de 50% après la première couche Dense.
  
Nous nous sommes inspirés de l’approche de [Keras pour le dataset MNIST](https://github.com/keras-team/keras/blob/keras-2/examples/mnist_cnn.py).

In [ ]:
model_with_dropout = create_model(use_dropout = True)

En procédant à l'entraînement de ce modèle comme nous l'avons vu auparavant, nous obtenons les graphiques suivants :

##### <center style="color:yellow;"> **Insérer graphiques** </center>

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests (avec le dataset du WKS2)**
- Apprend trop vite (Traning Accuracy > 95%) mais ne généralise pas mieux pour autant (Validation Loss en hausse)
- La Validation Accuracy se stabilise au bout de la 6ème époque
- Écart significatif entre la Traning Accuracy et la Validation Accuracy => surapprentissage
- Le Dropout seul semble ne pas suffire

##### <center style="color:yellow;"> **Insérer matrice de confusion** </center>

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

──────────────────────────────────────────────────

#### 10.3. <a id='early-stopping'>Early Stopping</a>

Pour cette approche, nous ajoutons un Early Stopping basé sur la perte de validation, avec une patience de 5 époques, afin d’interrompre l’entraînement dès que le modèle cesse de s’améliorer et de conserver les meilleurs poids.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    patience = 2,
    mode = 'max',
    restore_best_weights = True
)

callbacks.append(early_stopping)

En procédant à l'entraînement de ce modèle comme nous l'avons vu auparavant, nous obtenons les graphiques suivants :

##### <center style="color:yellow;"> **Insérer graphiques** </center>

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests (avec le dataset du WKS2)**
- Apprend trop vite (Traning Accuracy > 95%) mais ne généralise pas mieux pour autant
- Traning Accuracy proche de 100% à la fin + écart avec Validation Accuracy => surapprentissage
- La Validation Loss augmente, ce qui est un autre signe du surapprentissage
- Méthode insuffisante à elle seule pour prévenir l'overfitting

##### <center style="color:yellow;"> **Insérer matrice de confusion** </center>

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

---

### 11. <a id='final'>Modèle final</a>

<b style="color:yellow;">TODO</b>

Indiquer et justifier le choix du modèle final (modèle de base + data augmentation + dropout + early stopping) à l'aide des résultats observés. Faire la passe sur ses caractéristiques, à savoir :
- Paramètres
- Fonction de perte
- Algorithme d'optimisation utilisé pour l'entraînement

Inclure un schéma du modèle réalisé grâce à cet [outil](https://alexlenail.me/NN-SVG/LeNet.html).

---

### 12. <a id='conclusion'>Conclusion</a>

<b style="color:yellow;">TODO</b>

Écrire la conclusion.

---